In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta, time, datetime


import warnings
from IPython.core.display import display, HTML
warnings.filterwarnings("ignore") # suppress warnings because theyre annoying and i like living on the edge
pd.set_option('display.max_columns',100) # so i can see all columns
np.set_printoptions(linewidth = 220) # increase numpy array print
display(HTML("<style>.container { width:80% !important; }</style>")) # widen jupyter notebook blocks

In [2]:
conn_cred = {
    "dbservername":"localhost",
    "dbname":"main_schema",
    "dbuser":"minx",
    "dbpassword":"!xobILE!!!111!"
}

In [3]:
import pymysql as MySQLdb

In [4]:

from ibapi.wrapper import EWrapper
from ibapi.client import EClient
from ibapi.contract import Contract as IBcontract
from threading import Thread
import queue
import datetime

DEFAULT_HISTORIC_DATA_ID=50
DEFAULT_MARKET_DATA_ID=47
DEFAULT_GET_CONTRACT_ID=43

## marker for when queue is finished
FINISHED = object()
STARTED = object()
TIME_OUT = object()

class finishableQueue(object):

    def __init__(self, queue_to_finish):

        self._queue = queue_to_finish
        self.status = STARTED

    def get(self, timeout):
        """
        Returns a list of queue elements once timeout is finished, or a FINISHED flag is received in the queue
        :param timeout: how long to wait before giving up
        :return: list of queue elements
        """
        contents_of_queue=[]
        finished=False

        while not finished:
            try:
                current_element = self._queue.get(timeout=timeout)
                if current_element is FINISHED:
                    finished = True
                    self.status = FINISHED
                else:
                    contents_of_queue.append(current_element)
                    ## keep going and try and get more data

            except queue.Empty:
                ## If we hit a time out it's most probable we're not getting a finished element any time soon
                ## give up and return what we have
                finished = True
                self.status = TIME_OUT


        return contents_of_queue

    def timed_out(self):
        return self.status is TIME_OUT


def _nan_or_int(x):
    if not np.isnan(x):
        return int(x)
    else:
        return x

class stream_of_ticks(list):
    """
    Stream of ticks
    """

    def __init__(self, list_of_ticks):
        super().__init__(list_of_ticks)

    def as_pdDataFrame(self):

        if len(self)==0:
            ## no data; do a blank tick
            return tick(datetime.datetime.now()).as_pandas_row()

        pd_row_list=[tick.as_pandas_row() for tick in self]
        pd_data_frame=pd.concat(pd_row_list)

        return pd_data_frame


class tick(object):
    """
    Convenience method for storing ticks
    Not IB specific, use as abstract
    """
    def __init__(self, timestamp, bid_size=np.nan, bid_price=np.nan,
                 ask_size=np.nan, ask_price=np.nan,
                 last_trade_size=np.nan, last_trade_price=np.nan,
                 ignorable_tick_id=None):

        ## ignorable_tick_id keyword must match what is used in the IBtick class

        self.timestamp=timestamp
        self.bid_size=_nan_or_int(bid_size)
        self.bid_price=bid_price
        self.ask_size=_nan_or_int(ask_size)
        self.ask_price=ask_price
        self.last_trade_size=_nan_or_int(last_trade_size)
        self.last_trade_price=last_trade_price

    def __repr__(self):
        return self.as_pandas_row().__repr__()

    def as_pandas_row(self):
        """
        Tick as a pandas dataframe, single row, so we can concat together
        :return: pd.DataFrame
        """

        attributes=['bid_size','bid_price', 'ask_size', 'ask_price',
                    'last_trade_size', 'last_trade_price']

        self_as_dict=dict([(attr_name, getattr(self, attr_name)) for attr_name in attributes])

        return pd.DataFrame(self_as_dict, index=[self.timestamp])


class IBtick(tick):
    """
    Resolve IB tick categories
    """

    def __init__(self, timestamp, tickid, value):

        resolve_tickid=self.resolve_tickids(tickid)
        super().__init__(timestamp, **dict([(resolve_tickid, value)]))

    def resolve_tickids(self, tickid):

        tickid_dict=dict([("0", "bid_size"), ("1", "bid_price"), ("2", "ask_price"), ("3", "ask_size"),
                          ("4", "last_trade_price"), ("5", "last_trade_size")])

        if str(tickid) in tickid_dict.keys():
            return tickid_dict[str(tickid)]
        else:
            # This must be the same as the argument name in the parent class
            return "ignorable_tick_id"






class TestWrapper(EWrapper):
    """
    The wrapper deals with the action coming back from the IB gateway or TWS instance
    We override methods in EWrapper that will get called when this action happens, like currentTime
    Extra methods are added as we need to store the results in this object
    """

    def __init__(self):
        self._my_contract_details = {}
        self._my_historic_data_dict = {}
        self._my_market_data_dict = {}
        
    ## error handling code
    def init_error(self):
        error_queue=queue.Queue()
        self._my_errors = error_queue

    def get_error(self, timeout=5):
        if self.is_error():
            try:
                return self._my_errors.get(timeout=timeout)
            except queue.Empty:
                return None

        return None

    def is_error(self):
        an_error_if=not self._my_errors.empty()
        return an_error_if

    def error(self, id, errorCode, errorString):
        ## Overriden method
        errormsg = "IB error id %d errorcode %d string %s" % (id, errorCode, errorString)
        self._my_errors.put(errormsg)


    ## get contract details code
    def init_contractdetails(self, reqId):
        contract_details_queue = self._my_contract_details[reqId] = queue.Queue()

        return contract_details_queue

    def contractDetails(self, reqId, contractDetails):
        ## overridden method

        if reqId not in self._my_contract_details.keys():
            self.init_contractdetails(reqId)

        self._my_contract_details[reqId].put(contractDetails)

    def contractDetailsEnd(self, reqId):
        ## overriden method
        if reqId not in self._my_contract_details.keys():
            self.init_contractdetails(reqId)

        self._my_contract_details[reqId].put(FINISHED)

    ## Historic data code
    def init_historicprices(self, tickerid):
        historic_data_queue = self._my_historic_data_dict[tickerid] = queue.Queue()

        return historic_data_queue


    def historicalData(self, tickerid , bar):

        ## Overriden method
        ## Note I'm choosing to ignore barCount, WAP and hasGaps but you could use them if you like
        bardata=(bar.date, bar.open, bar.high, bar.low, bar.close, bar.volume)

        historic_data_dict=self._my_historic_data_dict

        ## Add on to the current data
        if tickerid not in historic_data_dict.keys():
            self.init_historicprices(tickerid)

        historic_data_dict[tickerid].put(bardata)

    def historicalDataEnd(self, tickerid, start:str, end:str):
        ## overriden method

        if tickerid not in self._my_historic_data_dict.keys():
            self.init_historicprices(tickerid)

        self._my_historic_data_dict[tickerid].put(FINISHED)

    ## market data

    def init_market_data(self, tickerid):
        market_data_queue = self._my_market_data_dict[tickerid] = queue.Queue()

        return market_data_queue


    def get_time_stamp(self):
        ## Time stamp to apply to market data
        ## We could also use IB server time
        return datetime.datetime.now()


    def tickPrice(self, tickerid , tickType, price, attrib):
        ##overriden method

        ## For simplicity I'm ignoring these but they could be useful to you...
        ## See the documentation http://interactivebrokers.github.io/tws-api/md_receive.html#gsc.tab=0
        # attrib.canAutoExecute
        # attrib.pastLimit

        this_tick_data=IBtick(self.get_time_stamp(),tickType, price)
        self._my_market_data_dict[tickerid].put(this_tick_data)


    def tickSize(self, tickerid, tickType, size):
        ## overriden method

        this_tick_data=IBtick(self.get_time_stamp(), tickType, size)
        self._my_market_data_dict[tickerid].put(this_tick_data)


    def tickString(self, tickerid, tickType, value):
        ## overriden method

        ## value is a string, make it a float, and then in the parent class will be resolved to int if size
        this_tick_data=IBtick(self.get_time_stamp(),tickType, float(value))
        self._my_market_data_dict[tickerid].put(this_tick_data)


    def tickGeneric(self, tickerid, tickType, value):
        ## overriden method

        this_tick_data=IBtick(self.get_time_stamp(),tickType, value)
        self._my_market_data_dict[tickerid].put(this_tick_data)

class TestClient(EClient):
    """
    The client method
    We don't override native methods, but instead call them from our own wrappers
    """
    def __init__(self, wrapper):
        ## Set up with a wrapper inside
        EClient.__init__(self, wrapper)
        self._market_data_q_dict = {}


    def resolve_ib_contract(self, ibcontract, reqId=DEFAULT_GET_CONTRACT_ID):

        """
        From a partially formed contract, returns a fully fledged version
        :returns fully resolved IB contract
        """

        ## Make a place to store the data we're going to return
        contract_details_queue = finishableQueue(self.init_contractdetails(reqId))

        print("Getting full contract details from the server... ")

        self.reqContractDetails(reqId, ibcontract)

        ## Run until we get a valid contract(s) or get bored waiting
        MAX_WAIT_SECONDS = 30
        new_contract_details = contract_details_queue.get(timeout = MAX_WAIT_SECONDS)

        while self.wrapper.is_error():
            print(self.get_error())

        if contract_details_queue.timed_out():
            print("Exceeded maximum wait for wrapper to confirm finished - seems to be normal behaviour")

        if len(new_contract_details)==0:
            print("Failed to get additional contract details: returning unresolved contract")
            return ibcontract

        if len(new_contract_details)>1:
            print("got multiple contracts using first one")

        new_contract_details=new_contract_details[0]

        resolved_ibcontract=new_contract_details.contract

        return resolved_ibcontract


    def get_IB_historical_data(self, ibcontract, durationStr="1 D", barSizeSetting="5 secs",
                               tickerid=DEFAULT_HISTORIC_DATA_ID):

        """
        Returns historical prices for a contract, up to today
        ibcontract is a Contract
        :returns list of prices in 4 tuples: Open high low close volume
        """


        ## Make a place to store the data we're going to return
        historic_data_queue = finishableQueue(self.init_historicprices(tickerid))

        # Request some historical data. Native method in EClient
        self.reqHistoricalData(
            tickerid,  # tickerId,
            ibcontract,  # contract,
            datetime.datetime.today().strftime("%Y%m%d %H:%M:%S %Z"),  # endDateTime,
            durationStr,  # durationStr,
            barSizeSetting,  # barSizeSetting,
            "TRADES",  # whatToShow,
            1,  # useRTH,
            1,  # formatDate
            False,  # KeepUpToDate <<==== added for api 9.73.2
            [] ## chartoptions not used
        )


        ## Wait until we get a completed data, an error, or get bored waiting
        MAX_WAIT_SECONDS = 200
        print("Getting historical data from the server... could take %d seconds to complete " % MAX_WAIT_SECONDS)

        historic_data = historic_data_queue.get(timeout = MAX_WAIT_SECONDS)

        while self.wrapper.is_error():
            print(self.get_error())

        if historic_data_queue.timed_out():
            print("Exceeded maximum wait for wrapper to confirm finished - seems to be normal behaviour")
        self.cancelHistoricalData(tickerid)


        return historic_data
    

class TestApp(TestWrapper, TestClient):
    def __init__(self, ipaddress, portid, clientid):
        error_queue=queue.Queue()
        self._my_errors = error_queue
        
        TestWrapper.__init__(self)
        TestClient.__init__(self, wrapper=self)

        self.connect(ipaddress, portid, clientid)

        thread = Thread(target = self.run)
        thread.start()

        setattr(self, "_thread", thread)

        self.init_error()


In [5]:
app = TestApp("127.0.0.1", 7497, 1)
ibcontract = IBcontract()
ibcontract.secType = "STK"
# ibcontract.lastTradeDateOrContractMonth="202011"
ibcontract.symbol="AAPL"
ibcontract.exchange="SMART"
ibcontract.currency = "USD"
ibcontract.primaryExchange = "NASDAQ"
resolved_ibcontract=app.resolve_ib_contract(ibcontract)
# print(resolved_ibcontract)
historic_data = app.get_IB_historical_data(resolved_ibcontract, durationStr = "1 D", barSizeSetting = "5 secs")


Getting full contract details from the server... 
IB error id -1 errorcode 2104 string Market data farm connection is OK:usfarm.nj
IB error id -1 errorcode 2104 string Market data farm connection is OK:cashfarm
IB error id -1 errorcode 2104 string Market data farm connection is OK:usfarm
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:euhmds
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:fundfarm
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:ushmds
IB error id -1 errorcode 2158 string Sec-def data farm connection is OK:secdefnj
Getting historical data from the server... could take 200 seconds to complete 


In [6]:
historic_data


[('20201106  06:30:00', 118.32, 118.42, 118.12, 118.37, 13667),
 ('20201106  06:30:05', 118.37, 118.42, 118.2, 118.28, 328),
 ('20201106  06:30:10', 118.29, 118.32, 118.18, 118.31, 782),
 ('20201106  06:30:15', 118.32, 118.32, 118.27, 118.32, 475),
 ('20201106  06:30:20', 118.32, 118.58, 118.28, 118.58, 1049),
 ('20201106  06:30:25', 118.58, 118.6, 118.51, 118.58, 564),
 ('20201106  06:30:30', 118.58, 118.62, 118.52, 118.59, 695),
 ('20201106  06:30:35', 118.59, 118.64, 118.53, 118.56, 702),
 ('20201106  06:30:40', 118.56, 118.6, 118.48, 118.5, 767),
 ('20201106  06:30:45', 118.5, 118.58, 118.45, 118.45, 461),
 ('20201106  06:30:50', 118.45, 118.5, 118.37, 118.5, 558),
 ('20201106  06:30:55', 118.51, 118.52, 118.26, 118.26, 353),
 ('20201106  06:31:00', 118.28, 118.49, 118.28, 118.41, 560),
 ('20201106  06:31:05', 118.41, 118.46, 118.36, 118.4, 450),
 ('20201106  06:31:10', 118.41, 118.48, 118.31, 118.33, 462),
 ('20201106  06:31:15', 118.34, 118.39, 118.27, 118.39, 400),
 ('20201106  

In [7]:
df = pd.DataFrame(historic_data, columns = ['date','open','high','low','close','volume'])

In [10]:
df['date'] = pd.to_datetime(df['date'],format = '%Y%m%d  %H:%M:%S'); df

date    open    high     low   close  volume
0    2020-11-06 06:30:00  118.32  118.42  118.12  118.37   13667
1    2020-11-06 06:30:05  118.37  118.42  118.20  118.28     328
2    2020-11-06 06:30:10  118.29  118.32  118.18  118.31     782
3    2020-11-06 06:30:15  118.32  118.32  118.27  118.32     475
4    2020-11-06 06:30:20  118.32  118.58  118.28  118.58    1049
...                  ...     ...     ...     ...     ...     ...
4675 2020-11-06 12:59:35  118.57  118.59  118.52  118.56     944
4676 2020-11-06 12:59:40  118.56  118.65  118.56  118.61     846
4677 2020-11-06 12:59:45  118.61  118.61  118.52  118.55     608
4678 2020-11-06 12:59:50  118.55  118.70  118.40  118.69    1971
4679 2020-11-06 12:59:55  118.69  118.73  118.64  118.69    1568

[4680 rows x 6 columns]

In [55]:
df['date'] = df['date'] + pd.DateOffset(hours=8); df

date    open    high     low   close  volume       epoch
0    2020-11-06 14:30:00  118.32  118.42  118.12  118.37   13667  1604644200
1    2020-11-06 14:30:05  118.37  118.42  118.20  118.28     328  1604644205
2    2020-11-06 14:30:10  118.29  118.32  118.18  118.31     782  1604644210
3    2020-11-06 14:30:15  118.32  118.32  118.27  118.32     475  1604644215
4    2020-11-06 14:30:20  118.32  118.58  118.28  118.58    1049  1604644220
...                  ...     ...     ...     ...     ...     ...         ...
4675 2020-11-06 20:59:35  118.57  118.59  118.52  118.56     944  1604667575
4676 2020-11-06 20:59:40  118.56  118.65  118.56  118.61     846  1604667580
4677 2020-11-06 20:59:45  118.61  118.61  118.52  118.55     608  1604667585
4678 2020-11-06 20:59:50  118.55  118.70  118.40  118.69    1971  1604667590
4679 2020-11-06 20:59:55  118.69  118.73  118.64  118.69    1568  1604667595

[4680 rows x 7 columns]

In [56]:
from datetime import datetime 
df['epoch'] = (df['date'] - datetime(1970,1,1)).dt.total_seconds(); df

date    open    high     low   close  volume         epoch
0    2020-11-06 14:30:00  118.32  118.42  118.12  118.37   13667  1.604673e+09
1    2020-11-06 14:30:05  118.37  118.42  118.20  118.28     328  1.604673e+09
2    2020-11-06 14:30:10  118.29  118.32  118.18  118.31     782  1.604673e+09
3    2020-11-06 14:30:15  118.32  118.32  118.27  118.32     475  1.604673e+09
4    2020-11-06 14:30:20  118.32  118.58  118.28  118.58    1049  1.604673e+09
...                  ...     ...     ...     ...     ...     ...           ...
4675 2020-11-06 20:59:35  118.57  118.59  118.52  118.56     944  1.604696e+09
4676 2020-11-06 20:59:40  118.56  118.65  118.56  118.61     846  1.604696e+09
4677 2020-11-06 20:59:45  118.61  118.61  118.52  118.55     608  1.604696e+09
4678 2020-11-06 20:59:50  118.55  118.70  118.40  118.69    1971  1.604696e+09
4679 2020-11-06 20:59:55  118.69  118.73  118.64  118.69    1568  1.604696e+09

[4680 rows x 7 columns]

In [57]:
list_vals = df[['epoch','open','high','low','close','volume']].values.tolist()

In [58]:
list_vals = list(map(lambda x: ['AAPL'] + x,list_vals))

In [59]:

def mysql_conn(db_conn_info):
    '''
    Call to connect to the database
    '''
    try:
        dbconn_ = MySQLdb.connect(host=db_conn_info['dbservername'],
                    db = db_conn_info['dbname'],
                    user = db_conn_info['dbuser'],
                    passwd = db_conn_info['dbpassword']
                    )
        cursor_ = dbconn_.cursor()
        return dbconn_, cursor_

    except Exception as e:
        print('Error: ' + str(e))

def db_conn_close(dbconn, cursor):
    # closing connections to free up sockets
    dbconn.commit()
    cursor.close()
    dbconn.close()
    
    
def run_query(creds, query):
    conn, cursor = mysql_conn(creds)

    try:
        cursor.execute(query)
    except Exception as e:
        print('Error in running query: ' + str(e))
    finally:
        db_conn_close(conn, cursor)

In [60]:
for i in range(len(list_vals)):
    query = "INSERT INTO {dbname}.bar_data (symbol, epoch,\
                open, high,low, close, volume\
                ) VALUES ({csv})".format(dbname = conn_cred['dbname'],
                                  csv = ','.join(map(lambda x: "'" + str(x) + "'",list_vals[i])))
    run_query(conn_cred, query)

In [47]:
','.join(map(lambda x: "'" + str(x) + "'",list_vals[0]))

"'AAPL','1604644200.0','118.32','118.42','118.12','118.37','13667.0'"